# Scrapy 爬虫框架学习教程

## 目录
1. [Scrapy简介](#scrapy简介)
2. [环境准备](#环境准备)
3. [第一个Scrapy爬虫](#第一个scrapy爬虫)
4. [选择器（Selectors）详解](#选择器详解)
5. [数据提取与Item](#数据提取与item)
6. [请求与响应处理](#请求与响应处理)
7. [爬虫进阶：中间件和管道](#爬虫进阶)
8. [HTTP协议与网络安全](#http协议与网络安全)
9. [合规爬虫实践](#合规爬虫实践)
10. [实践练习](#实践练习)


## Scrapy简介

**Scrapy** 是一个用于爬取网站数据并提取结构化数据的Python框架。它是一个功能强大、可扩展的爬虫框架。

### 计算机网络基础：爬虫工作原理

想象一下，爬虫就像是一个**自动化的浏览器**：

1. **发送HTTP请求**：爬虫向目标网站发送HTTP请求（就像你在浏览器地址栏输入网址）
2. **接收HTTP响应**：服务器返回HTML、JSON等格式的数据（就像浏览器显示网页）
3. **解析数据**：从响应中提取需要的信息（就像你阅读网页内容）
4. **存储数据**：将提取的数据保存到文件或数据库

### HTTP请求流程（形象化描述）

```
客户端（爬虫）                   服务器（网站）
    |                                |
    |--- HTTP请求 (GET/POST) ------->|
    |   (包含URL、Headers等信息)      |
    |                                |
    |<-- HTTP响应 (200 OK) ----------|
    |   (包含HTML内容、状态码等)      |
    |                                |
    |--- 解析HTML内容 --------------|
    |--- 提取数据 ------------------|
    |--- 存储数据 ------------------|
```

### Scrapy的主要特点
- **高性能**：异步处理，可以并发爬取多个页面
- **可扩展**：通过中间件和管道轻松扩展功能
- **内置选择器**：支持CSS选择器和XPath
- **自动去重**：自动处理重复请求
- **遵守robots.txt**：可以配置遵守网站的爬虫协议


## 环境准备

### 安装Scrapy

在命令行中执行：
```bash
pip install scrapy
```

### 验证安装


In [ ]:
# 导入scrapy库并查看版本
import scrapy

print(f"Scrapy版本: {scrapy.__version__}")

# 检查scrapy命令是否可用（在命令行中运行：scrapy --version）
# 注意：在Jupyter Notebook中，某些scrapy命令需要在终端执行


## 第一个Scrapy爬虫

### 创建Scrapy项目（在命令行中执行）

```bash
# 创建项目
scrapy startproject myproject

# 进入项目目录
cd myproject

# 创建爬虫
scrapy genspider example example.com
```

### 在Jupyter Notebook中直接使用Scrapy

由于Jupyter Notebook的特殊环境，我们可以直接编写爬虫代码来演示：


In [ ]:
# 示例1：最简单的Scrapy爬虫
# 这个示例演示了如何创建一个基本的爬虫类

from scrapy import Spider
from scrapy.http import Request

class SimpleSpider(Spider):
    """
    Spider是Scrapy中所有爬虫的基类
    
    关键概念：
    - name: 爬虫的唯一标识符，用于在命令行中运行爬虫
    - start_urls: 爬虫开始爬取的URL列表
    - parse(): 默认的回调函数，处理start_urls返回的响应
    """
    name = 'simple'  # 爬虫名称，必须唯一
    
    # 起始URL列表，爬虫会从这些URL开始爬取
    # 注意：这里使用httpbin.org作为示例，这是一个用于测试HTTP请求的网站
    start_urls = [
        'https://httpbin.org/html',  # 返回一个简单的HTML页面
    ]
    
    def parse(self, response):
        """
        parse方法是处理响应的回调函数
        
        参数：
        - response: Response对象，包含服务器返回的HTML内容
        
        返回值：
        - 可以返回Item对象（数据）、Request对象（继续爬取）或None
        """
        # response.text 包含响应的文本内容（HTML）
        # 这里我们只打印标题
        title = response.css('title::text').get()
        print(f"页面标题: {title}")
        
        # 返回None表示这个请求处理完成
        return None


## 选择器（Selectors）详解

Scrapy提供了强大的选择器来从HTML中提取数据，支持**CSS选择器**和**XPath**两种方式。

### CSS选择器 vs XPath

- **CSS选择器**：语法简洁，类似jQuery选择器，适合简单的选择
- **XPath**：功能更强大，可以基于XML路径表达式选择元素，适合复杂场景

### 选择器方法

- `response.css('selector')` - 使用CSS选择器
- `response.xpath('xpath')` - 使用XPath选择器
- `.get()` - 获取第一个匹配的元素（返回字符串）
- `.getall()` - 获取所有匹配的元素（返回列表）


In [ ]:
# 示例2：使用选择器提取数据
# 这个示例演示了CSS选择器和XPath的基本用法

from scrapy import Spider
from scrapy.selector import Selector

# 创建一个示例HTML内容用于演示
sample_html = """
<html>
<head>
    <title>示例网页</title>
</head>
<body>
    <div class="container">
        <h1>欢迎学习Scrapy</h1>
        <ul class="items">
            <li class="item">项目1</li>
            <li class="item">项目2</li>
            <li class="item">项目3</li>
        </ul>
        <div id="content">
            <p>这是第一段内容</p>
            <p>这是第二段内容</p>
        </div>
        <a href="https://example.com">链接</a>
    </div>
</body>
</html>
"""

# 创建Selector对象
sel = Selector(text=sample_html)

print("=== CSS选择器示例 ===")
# CSS选择器：选择title标签的文本内容
title = sel.css('title::text').get()
print(f"标题: {title}")

# CSS选择器：选择所有class为item的li元素
items = sel.css('li.item::text').getall()
print(f"项目列表: {items}")

# CSS选择器：选择id为content的div下的所有p标签
paragraphs = sel.css('#content p::text').getall()
print(f"段落内容: {paragraphs}")

# CSS选择器：提取链接的href属性
link = sel.css('a::attr(href)').get()
print(f"链接地址: {link}")

print("\n=== XPath选择器示例 ===")
# XPath：选择title标签的文本内容
title_xpath = sel.xpath('//title/text()').get()
print(f"标题(XPath): {title_xpath}")

# XPath：选择所有class为item的li元素
items_xpath = sel.xpath('//li[@class="item"]/text()').getall()
print(f"项目列表(XPath): {items_xpath}")

# XPath：选择id为content的div下的所有p标签
paragraphs_xpath = sel.xpath('//div[@id="content"]/p/text()').getall()
print(f"段落内容(XPath): {paragraphs_xpath}")

# XPath：提取链接的href属性
link_xpath = sel.xpath('//a/@href').get()
print(f"链接地址(XPath): {link_xpath}")

print("\n=== 选择器语法说明 ===")
print("""
CSS选择器语法：
- 'tag' : 选择标签
- '.class' : 选择class属性
- '#id' : 选择id属性
- '::text' : 提取文本内容
- '::attr(attribute)' : 提取属性值

XPath语法：
- '//tag' : 选择所有tag标签（任意位置）
- '/tag' : 选择直接子元素
- '[@attribute="value"]' : 属性选择
- '/text()' : 提取文本内容
- '/@attribute' : 提取属性值
""")


## 数据提取与Item

### Item：结构化数据容器

在Scrapy中，**Item**用于定义要提取的数据结构。它类似于Python的字典，但提供了字段定义和验证功能。

### Item的优势

- **类型验证**：可以定义字段类型，自动验证数据
- **代码提示**：IDE可以提供更好的代码补全
- **文档化**：字段定义本身就是文档


In [ ]:
# 示例3：使用Item定义数据结构

from scrapy import Item, Field

class BookItem(Item):
    """
    Item类用于定义要提取的数据结构
    
    概念说明：
    - Item类似于字典，但提供了字段定义
    - Field()用于定义字段，可以设置元数据（如序列化器、验证器等）
    - 字段名必须是字符串
    """
    # 定义字段：字段名 = Field()
    title = Field()      # 书名
    author = Field()     # 作者
    price = Field()      # 价格
    url = Field()        # 链接
    description = Field()  # 描述（可选字段）

# 创建Item实例
book = BookItem()
book['title'] = 'Python编程'
book['author'] = '作者名'
book['price'] = '99.00'
book['url'] = 'https://example.com/book1'

print("Item内容:")
print(dict(book))  # 转换为字典查看

# Item也可以像字典一样访问
print(f"\n书名: {book['title']}")
print(f"作者: {book.get('author', '未知')}")  # 使用get方法，如果不存在返回默认值


In [ ]:
# 示例4：完整的爬虫示例 - 提取数据到Item

from scrapy import Spider, Item, Field
from scrapy.http import Request

# 定义Item（在实际项目中，通常在items.py文件中定义）
class NewsItem(Item):
    title = Field()
    content = Field()
    publish_date = Field()
    author = Field()

class NewsSpider(Spider):
    """
    新闻爬虫示例
    
    这个爬虫演示了：
    1. 如何从HTML中提取数据
    2. 如何将数据填充到Item中
    3. 如何返回Item对象
    """
    name = 'news'
    
    # 注意：这里使用httpbin.org作为测试，实际项目中应该使用真实的新闻网站
    # 合规提醒：爬取真实网站前，请务必：
    # 1. 查看网站的robots.txt文件
    # 2. 阅读网站的服务条款
    # 3. 遵守网站的爬取频率限制
    # 4. 不要对服务器造成过大压力
    start_urls = [
        'https://httpbin.org/html',
    ]
    
    def parse(self, response):
        """
        解析响应并提取数据
        
        工作流程：
        1. 使用选择器从HTML中提取数据
        2. 创建Item实例
        3. 将提取的数据填充到Item中
        4. 返回Item（Scrapy会自动处理Item，发送到Pipeline）
        """
        # 创建Item实例
        item = NewsItem()
        
        # 使用CSS选择器提取数据
        # 注意：httpbin.org/html返回的页面结构简单，这里仅作演示
        item['title'] = response.css('title::text').get() or '无标题'
        item['content'] = response.css('h1::text').get() or '无内容'
        
        # 如果没有找到数据，可以设置默认值
        item['publish_date'] = '2024-01-01'  # 示例数据
        item['author'] = '未知作者'  # 示例数据
        
        # 返回Item，Scrapy会将Item发送到Pipeline进行处理
        # 在Pipeline中可以保存到文件、数据库等
        yield item
        
        # yield关键字说明：
        # - yield是Python的生成器关键字
        # - 在Scrapy中，yield可以返回Item或Request
        # - 使用yield可以让Scrapy异步处理多个Item/Request
        # - 相比return，yield不会立即结束函数，可以继续处理后续数据


## 请求与响应处理

### 发送自定义请求

在实际爬虫中，我们经常需要：
- 发送POST请求（而不是默认的GET）
- 添加自定义Headers（如User-Agent、Cookie等）
- 传递参数（表单数据、JSON数据等）
- 处理重定向、Cookie等

### HTTP请求方法

- **GET**：获取资源，参数在URL中（如：`?key=value`）
- **POST**：提交数据，数据在请求体中（如：表单提交、API调用）
- **HEAD**：只获取响应头，不获取响应体
- **PUT/DELETE**：用于RESTful API


In [ ]:
# 示例5：发送自定义请求

from scrapy import Spider, Request
from scrapy.http import FormRequest

class CustomRequestSpider(Spider):
    """
    演示如何发送自定义请求
    """
    name = 'custom_request'
    
    def start_requests(self):
        """
        重写start_requests方法，可以自定义初始请求
        
        与start_urls的区别：
        - start_urls：简单场景，只能设置URL列表
        - start_requests：复杂场景，可以设置Headers、Cookies、回调函数等
        """
        # 方法1：使用Request对象，可以设置更多参数
        yield Request(
            url='https://httpbin.org/get',
            method='GET',
            headers={
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36',
                # User-Agent说明：
                # - 标识客户端（浏览器/爬虫）的类型和版本
                # - 有些网站会根据User-Agent返回不同的内容
                # - 合规提醒：不要伪装成其他知名浏览器，保持真实身份
            },
            callback=self.parse_get,  # 指定处理响应的回调函数
            meta={'page': 1}  # 传递额外的元数据
        )
        
        # 方法2：发送POST请求（带表单数据）
        yield FormRequest(
            url='https://httpbin.org/post',
            formdata={
                'username': 'test',
                'password': 'test123'
            },
            callback=self.parse_post,
            # 合规提醒：不要爬取需要登录的网站，除非获得明确授权
        )
    
    def parse_get(self, response):
        """
        处理GET请求的响应
        """
        # 访问meta中传递的数据
        page = response.meta.get('page', 1)
        print(f"处理第 {page} 页")
        
        # httpbin.org/get 会返回请求的信息（JSON格式）
        import json
        data = json.loads(response.text)
        print(f"请求URL: {data.get('url')}")
        print(f"User-Agent: {data.get('headers', {}).get('User-Agent')}")
    
    def parse_post(self, response):
        """
        处理POST请求的响应
        """
        import json
        data = json.loads(response.text)
        print(f"POST数据: {data.get('form')}")
        
        # 合规提醒：POST请求通常用于提交数据
        # 不要向网站提交恶意数据或进行自动化攻击


In [ ]:
# 示例6：处理分页和链接跟踪

from scrapy import Spider, Request

class PaginationSpider(Spider):
    """
    演示如何处理分页和跟踪链接
    """
    name = 'pagination'
    
    start_urls = ['https://httpbin.org/html']
    
    def parse(self, response):
        """
        解析页面并跟踪链接
        """
        # 提取当前页面的数据
        title = response.css('title::text').get()
        print(f"当前页面: {title}")
        
        # 示例：假设我们要跟踪页面中的链接
        # 在实际场景中，可能是"下一页"链接或文章列表中的链接
        links = response.css('a::attr(href)').getall()
        
        # 过滤和构建绝对URL
        for link in links[:3]:  # 只处理前3个链接作为示例
            if link:
                # urljoin：将相对URL转换为绝对URL
                # 例如：'/page2' + 'https://example.com/page1' = 'https://example.com/page2'
                absolute_url = response.urljoin(link)
                
                # 创建新的Request，继续爬取
                yield Request(
                    url=absolute_url,
                    callback=self.parse_detail,  # 使用不同的回调函数处理详情页
                    # 注意：Scrapy会自动去重，相同的URL只会请求一次
                )
    
    def parse_detail(self, response):
        """
        处理详情页
        """
        print(f"处理详情页: {response.url}")
        
        # 可以继续提取数据或跟踪更多链接
        # 注意：要设置合理的深度限制，避免无限爬取


## 爬虫进阶：中间件和管道

### Scrapy架构（形象化描述）

```
┌─────────────────────────────────────────┐
│          Scrapy Engine (引擎)            │
│  (协调所有组件，处理请求和响应)           │
┌─────────────────────────────────────────┐
│                                         │
│  ┌──────────┐      ┌──────────┐        │
│  │ Scheduler│ <──> │ Downloader│       │
│  │ (调度器) │      │ (下载器) │        │
│  └──────────┘      └──────────┘        │
│       │                  │              │
│       │                  ▼              │
│       │         ┌─────────────────┐    │
│       │         │ Downloader      │    │
│       │         │ Middlewares     │    │
│       │         │ (下载中间件)    │    │
│       │         └─────────────────┘    │
│       │                  │              │
│       ▼                  ▼              │
│  ┌──────────┐      ┌──────────┐        │
│  │  Spider  │ ───> │ Response │        │
│  │ (爬虫)   │      │ (响应)   │        │
│  └──────────┘      └──────────┘        │
│       │                                 │
│       ▼                                 │
│  ┌──────────┐                          │
│  │   Item   │                          │
│  └──────────┘                          │
│       │                                 │
│       ▼                                 │
│  ┌─────────────────┐                   │
│  │ Item Pipeline   │                   │
│  │ (数据处理管道)   │                   │
│  └─────────────────┘                   │
└─────────────────────────────────────────┘
```

### 关键组件说明

1. **Spider（爬虫）**：定义如何爬取网站，包括起始URL和解析逻辑
2. **Scheduler（调度器）**：决定下一个要爬取的请求
3. **Downloader（下载器）**：实际发送HTTP请求并获取响应
4. **Middleware（中间件）**：在请求/响应处理过程中进行预处理
5. **Pipeline（管道）**：处理Spider返回的Item（清洗、验证、存储）


In [ ]:
# 示例7：自定义Pipeline处理数据

from scrapy import Spider, Item, Field
import json
import csv

# 定义Item
class ProductItem(Item):
    name = Field()
    price = Field()
    category = Field()

# Pipeline示例1：保存为JSON文件
class JsonPipeline:
    """
    Pipeline用于处理Spider返回的Item
    
    工作流程：
    1. process_item()：处理每个Item
    2. open_spider()：爬虫开始时调用（初始化）
    3. close_spider()：爬虫结束时调用（清理）
    """
    
    def open_spider(self, spider):
        """
        爬虫开始时调用
        用于初始化资源（如打开文件、连接数据库）
        """
        self.file = open('products.json', 'w', encoding='utf-8')
        self.items = []
        print("Pipeline: 打开文件")
    
    def close_spider(self, spider):
        """
        爬虫结束时调用
        用于清理资源（如关闭文件、断开数据库连接）
        """
        json.dump(self.items, self.file, ensure_ascii=False, indent=2)
        self.file.close()
        print(f"Pipeline: 保存了 {len(self.items)} 条数据到 products.json")
    
    def process_item(self, item, spider):
        """
        处理每个Item
        
        参数：
        - item: Spider返回的Item对象
        - spider: 当前运行的Spider实例
        
        返回值：
        - 必须返回Item对象（或DropItem异常）
        """
        # 将Item转换为字典并添加到列表
        self.items.append(dict(item))
        
        # 可以在这里进行数据清洗、验证等操作
        # 例如：去除空白字符、验证数据格式等
        
        return item  # 必须返回Item，否则后续Pipeline无法处理

# Pipeline示例2：保存为CSV文件
class CsvPipeline:
    def open_spider(self, spider):
        self.file = open('products.csv', 'w', newline='', encoding='utf-8')
        self.writer = None
        self.fieldnames = ['name', 'price', 'category']
        print("CSV Pipeline: 打开文件")
    
    def close_spider(self, spider):
        if self.file:
            self.file.close()
        print("CSV Pipeline: 文件已关闭")
    
    def process_item(self, item, spider):
        if self.writer is None:
            # 第一次调用时，创建CSV写入器并写入表头
            self.writer = csv.DictWriter(self.file, fieldnames=self.fieldnames)
            self.writer.writeheader()
        
        # 写入数据行
        self.writer.writerow(dict(item))
        return item

# 使用Pipeline的Spider示例
class ProductSpider(Spider):
    name = 'product'
    start_urls = ['https://httpbin.org/html']
    
    def parse(self, response):
        # 创建示例Item
        item = ProductItem()
        item['name'] = '示例商品'
        item['price'] = '99.00'
        item['category'] = '电子产品'
        
        yield item  # Item会被发送到Pipeline处理

# 注意：在实际项目中，Pipeline需要在settings.py中配置：
# ITEM_PIPELINES = {
#     'myproject.pipelines.JsonPipeline': 300,  # 数字表示优先级，越小越先执行
#     'myproject.pipelines.CsvPipeline': 400,
# }


## HTTP协议与网络安全

### HTTP协议基础（形象化描述）

HTTP（HyperText Transfer Protocol）就像**邮递员送信**的过程：

```
客户端（浏览器/爬虫）               服务器（网站）
    |                                    |
    |--- 1. 写请求信 ------------------->|
    |     (请求行：GET /page HTTP/1.1)   |
    |     (请求头：Host, User-Agent等)   |
    |     (请求体：POST数据，如果有)     |
    |                                    |
    |<-- 2. 收到回信 --------------------|
    |     (状态行：HTTP/1.1 200 OK)      |
    |     (响应头：Content-Type等)       |
    |     (响应体：HTML/JSON内容)        |
    |                                    |
```

### HTTP状态码（重要概念）

- **200 OK**：请求成功
- **301/302**：重定向（网站搬家了，告诉你新地址）
- **404 Not Found**：页面不存在
- **403 Forbidden**：禁止访问（可能被反爬虫机制拦截）
- **500 Internal Server Error**：服务器错误
- **503 Service Unavailable**：服务不可用（可能服务器过载）

### HTTP请求头（Headers）详解

请求头就像**信封上的信息**，告诉服务器关于请求的详细信息：

- **User-Agent**：客户端标识（浏览器类型、操作系统等）
- **Referer**：来源页面（从哪个页面跳转过来的）
- **Cookie**：会话信息（登录状态、购物车等）
- **Accept**：客户端能接受的内容类型
- **Content-Type**：请求体的内容类型（POST请求时使用）


In [ ]:
# 示例8：理解HTTP请求和响应

from scrapy.http import Request, Response
from scrapy import Spider

class HttpUnderstandingSpider(Spider):
    """
    演示HTTP请求和响应的各个组成部分
    """
    name = 'http_demo'
    
    def start_requests(self):
        # 创建一个详细的请求
        request = Request(
            url='https://httpbin.org/headers',  # httpbin.org可以返回请求的详细信息
            headers={
                'User-Agent': 'MyScrapyBot/1.0',
                'Accept': 'text/html,application/json',
                'Referer': 'https://example.com',
            }
        )
        
        print("=== HTTP请求信息 ===")
        print(f"请求方法: {request.method}")
        print(f"请求URL: {request.url}")
        print(f"请求头: {dict(request.headers)}")
        print(f"请求体: {request.body}")
        
        yield request
    
    def parse(self, response):
        print("\n=== HTTP响应信息 ===")
        print(f"状态码: {response.status}")
        print(f"响应URL: {response.url}")
        print(f"响应头: {dict(response.headers)}")
        print(f"响应体长度: {len(response.body)} 字节")
        
        # httpbin.org/headers 会返回我们发送的请求头
        import json
        try:
            data = json.loads(response.text)
            print(f"\n服务器收到的请求头:")
            print(json.dumps(data.get('headers', {}), indent=2, ensure_ascii=False))
        except:
            print("响应内容:", response.text[:200])  # 只显示前200个字符
        
        # 状态码处理示例
        if response.status == 200:
            print("\n✓ 请求成功")
        elif response.status == 404:
            print("\n✗ 页面不存在")
        elif response.status == 403:
            print("\n✗ 访问被禁止（可能触发了反爬虫机制）")
        elif response.status >= 500:
            print("\n✗ 服务器错误")


### 网络安全问题：反爬虫机制

网站为了保护自己，会使用各种**反爬虫机制**：

#### 1. User-Agent检测
- **原理**：检查请求头中的User-Agent
- **应对**：设置合理的User-Agent（但不要伪装成知名浏览器）

#### 2. 频率限制（Rate Limiting）
- **原理**：限制同一IP的请求频率
- **形象化**：就像银行ATM，同一张卡短时间内只能取款有限次数
- **应对**：设置下载延迟（DOWNLOAD_DELAY）

#### 3. IP封禁
- **原理**：检测到异常行为后封禁IP地址
- **应对**：使用代理IP（但需要遵守代理服务商的使用条款）

#### 4. Cookie/Session验证
- **原理**：需要登录或验证身份才能访问
- **合规提醒**：不要尝试绕过登录验证，这通常违反服务条款

#### 5. JavaScript渲染
- **原理**：内容通过JavaScript动态加载
- **应对**：使用Selenium或Splash（但会增加复杂度）

#### 6. 验证码（CAPTCHA）
- **原理**：要求人类验证
- **合规提醒**：不要使用自动化工具绕过验证码


In [ ]:
# 示例9：设置合理的爬取参数（避免被封禁）

from scrapy import Spider
from scrapy.settings import Settings

class PoliteSpider(Spider):
    """
    演示如何设置合理的爬取参数，做一个"有礼貌"的爬虫
    """
    name = 'polite'
    
    # 自定义设置
    custom_settings = {
        # 下载延迟：每次请求之间等待1-3秒（随机）
        # 形象化：就像人类浏览网页，不会瞬间点击所有链接
        'DOWNLOAD_DELAY': 2,  # 固定延迟2秒
        'RANDOMIZE_DOWNLOAD_DELAY': True,  # 随机延迟（0.5 * DOWNLOAD_DELAY 到 1.5 * DOWNLOAD_DELAY）
        
        # 并发请求数：同时进行的请求数量
        # 形象化：就像同时打开多个浏览器标签页
        'CONCURRENT_REQUESTS': 16,  # 默认16个
        'CONCURRENT_REQUESTS_PER_DOMAIN': 8,  # 每个域名最多8个并发
        
        # User-Agent：设置合理的用户代理
        'USER_AGENT': 'MyScrapyBot/1.0 (+https://example.com/bot-info)',
        # 合规提醒：提供bot信息页面，说明爬虫的目的和联系方式
        
        # 遵守robots.txt
        'ROBOTSTXT_OBEY': True,
        # robots.txt说明：
        # - 网站根目录下的文件，告诉爬虫哪些可以爬，哪些不能爬
        # - 例如：User-agent: *  Disallow: /admin/
        # - 遵守robots.txt是网络爬虫的基本礼仪
        
        # 自动重试
        'RETRY_ENABLED': True,
        'RETRY_TIMES': 3,  # 失败后重试3次
        'RETRY_HTTP_CODES': [500, 502, 503, 504, 408, 429],  # 这些状态码会重试
        
        # 超时设置
        'DOWNLOAD_TIMEOUT': 180,  # 180秒超时
    }
    
    start_urls = ['https://httpbin.org/delay/1']  # 模拟延迟1秒的请求
    
    def parse(self, response):
        print("请求成功完成")
        print(f"响应时间: {response.meta.get('download_latency', 0):.2f} 秒")
        
        # 合规提醒总结：
        # 1. 设置合理的延迟，不要对服务器造成压力
        # 2. 遵守robots.txt协议
        # 3. 设置合理的User-Agent，不要伪装
        # 4. 不要绕过网站的访问限制
        # 5. 尊重网站的服务条款


## 合规爬虫实践

### 法律和道德考量

网络爬虫虽然技术上是可行的，但必须遵守**法律法规**和**道德规范**：

#### 1. 遵守robots.txt协议

**robots.txt**是网站告诉爬虫的"交通规则"：

```
User-agent: *          # 适用于所有爬虫
Allow: /public/        # 允许爬取/public/目录
Disallow: /private/    # 禁止爬取/private/目录
Disallow: /admin/      # 禁止爬取/admin/目录
Crawl-delay: 10         # 请求间隔至少10秒
```

**合规做法**：
- 在Scrapy中设置 `ROBOTSTXT_OBEY = True`
- 遵守网站设置的爬取规则

#### 2. 遵守服务条款（Terms of Service）

**重要**：大多数网站的服务条款都禁止：
- 自动化访问（除非明确允许）
- 大规模数据抓取
- 商业用途的数据使用

**合规做法**：
- 爬取前阅读网站的服务条款
- 如有疑问，联系网站管理员获得授权
- 仅爬取公开数据，不爬取需要登录的内容

#### 3. 数据使用限制

**合规做法**：
- **个人学习/研究**：通常可以接受
- **商业用途**：需要获得明确授权
- **个人隐私数据**：严格禁止（如用户个人信息、联系方式等）
- **版权内容**：遵守版权法，不要大规模复制受版权保护的内容

#### 4. 技术合规

**合规做法**：
- 设置合理的请求频率（不要DDoS攻击）
- 不要绕过安全措施（如验证码、登录验证）
- 不要伪装成其他服务（如伪装成Googlebot）
- 提供联系方式（在User-Agent中）

#### 5. 数据存储和使用

**合规做法**：
- 仅存储必要的数据
- 遵守数据保护法规（如GDPR）
- 不要将数据用于非法用途
- 定期清理不需要的数据


In [ ]:
# 示例10：合规爬虫的最佳实践

from scrapy import Spider, Request
from scrapy.utils.response import get_base_url

class CompliantSpider(Spider):
    """
    演示合规爬虫的最佳实践
    """
    name = 'compliant'
    
    custom_settings = {
        # 1. 遵守robots.txt
        'ROBOTSTXT_OBEY': True,
        
        # 2. 设置合理的延迟
        'DOWNLOAD_DELAY': 2,
        'RANDOMIZE_DOWNLOAD_DELAY': True,
        
        # 3. 设置合理的User-Agent（包含联系方式）
        'USER_AGENT': 'MyResearchBot/1.0 (Contact: researcher@example.com; Purpose: Academic Research)',
        
        # 4. 限制并发数
        'CONCURRENT_REQUESTS_PER_DOMAIN': 2,
        
        # 5. 设置超时
        'DOWNLOAD_TIMEOUT': 30,
        
        # 6. 自动重试（但不要无限重试）
        'RETRY_ENABLED': True,
        'RETRY_TIMES': 2,
    }
    
    def start_requests(self):
        """
        在发送请求前，可以进行合规性检查
        """
        # 示例：检查是否应该爬取某个URL
        url = 'https://httpbin.org/html'
        
        # 合规检查示例：
        # 1. 检查URL是否在允许的域名列表中
        # 2. 检查是否需要登录（如果需要，应该停止）
        # 3. 检查是否违反了robots.txt
        
        yield Request(
            url=url,
            callback=self.parse,
            errback=self.handle_error,  # 错误处理回调
            dont_filter=False,  # 使用默认的去重机制
        )
    
    def parse(self, response):
        """
        解析响应
        """
        # 合规检查：检查响应状态
        if response.status == 403:
            self.logger.warning("收到403错误，可能触发了反爬虫机制，建议停止爬取")
            return
        
        if response.status == 429:
            self.logger.warning("收到429错误（请求过多），建议增加延迟时间")
            return
        
        # 提取数据
        title = response.css('title::text').get()
        print(f"页面标题: {title}")
        
        # 合规提醒：
        # - 只提取公开可见的数据
        # - 不要提取个人隐私信息
        # - 不要提取受版权保护的内容
    
    def handle_error(self, failure):
        """
        错误处理
        """
        # 记录错误，但不要无限重试
        self.logger.error(f"请求失败: {failure.value}")
        
        # 合规做法：遇到错误时，应该：
        # 1. 记录错误日志
        # 2. 不要立即重试（避免对服务器造成压力）
        # 3. 分析错误原因（是否是合规问题）

# 合规检查清单函数
def compliance_checklist():
    """
    爬虫开发前的合规检查清单
    """
    checklist = """
    ✓ 合规爬虫检查清单：
    
    1. 法律合规
       [ ] 阅读并理解目标网站的服务条款
       [ ] 确认爬取行为不违反法律法规
       [ ] 确认数据使用目的合法
    
    2. 技术合规
       [ ] 设置ROBOTSTXT_OBEY = True
       [ ] 设置合理的DOWNLOAD_DELAY
       [ ] 设置合理的User-Agent（包含联系方式）
       [ ] 限制并发请求数
       [ ] 实现错误处理和日志记录
    
    3. 数据合规
       [ ] 仅爬取公开数据
       [ ] 不爬取个人隐私信息
       [ ] 不爬取需要登录的内容（除非获得授权）
       [ ] 遵守数据保护法规
    
    4. 道德合规
       [ ] 不对服务器造成过大压力
       [ ] 尊重网站的反爬虫机制
       [ ] 提供明确的爬虫身份和目的
       [ ] 在遇到问题时主动联系网站管理员
    """
    print(checklist)

# 运行检查清单
compliance_checklist()


## 实践练习

### 练习1：基础爬虫

创建一个爬虫，从 httpbin.org/html 提取页面标题和所有链接。

### 练习2：数据提取

创建一个Item类，包含以下字段：
- title（标题）
- links（链接列表）
- text_content（文本内容）

### 练习3：处理多个页面

创建一个爬虫，能够：
1. 从起始页面提取链接
2. 跟踪链接到详情页
3. 从详情页提取数据

### 练习4：数据存储

创建一个Pipeline，将数据保存为：
- JSON格式
- CSV格式

### 练习5：错误处理

实现错误处理逻辑：
- 处理404错误
- 处理403错误
- 处理超时错误


In [ ]:
# 练习1和2的示例代码

from scrapy import Spider, Item, Field

# 定义Item
class PageItem(Item):
    title = Field()
    links = Field()  # 列表类型
    text_content = Field()

class PracticeSpider(Spider):
    """
    练习：基础爬虫和数据提取
    """
    name = 'practice'
    start_urls = ['https://httpbin.org/html']
    
    def parse(self, response):
        # 创建Item
        item = PageItem()
        
        # 提取标题
        item['title'] = response.css('title::text').get()
        
        # 提取所有链接（使用getall获取列表）
        item['links'] = response.css('a::attr(href)').getall()
        
        # 提取文本内容（去除HTML标签）
        # 注意：这里只是简单示例，实际可能需要更复杂的处理
        item['text_content'] = ' '.join(response.css('body *::text').getall())
        
        # 打印结果
        print(f"标题: {item['title']}")
        print(f"链接数量: {len(item['links'])}")
        print(f"文本内容长度: {len(item['text_content'])} 字符")
        
        return item

# 注意：在实际运行中，需要使用scrapy crawl命令：
# scrapy crawl practice -o output.json


## 总结

### 关键知识点回顾

1. **Scrapy架构**：理解Spider、Scheduler、Downloader、Pipeline等组件的作用
2. **选择器**：掌握CSS选择器和XPath的基本用法
3. **Item**：使用Item定义数据结构，提高代码可维护性
4. **请求处理**：理解HTTP请求和响应的各个组成部分
5. **Pipeline**：使用Pipeline处理、清洗和存储数据
6. **合规实践**：遵守robots.txt、设置合理延迟、尊重网站规则

### 进一步学习方向

- **Scrapy官方文档**：https://docs.scrapy.org/
- **XPath教程**：学习更复杂的XPath表达式
- **中间件开发**：自定义下载中间件和爬虫中间件
- **分布式爬虫**：使用Scrapy-Redis实现分布式爬取
- **JavaScript渲染**：使用Splash处理动态内容

### 重要提醒

⚠️ **合规第一**：在开发任何爬虫之前，务必：
1. 阅读目标网站的服务条款
2. 检查robots.txt文件
3. 设置合理的爬取参数
4. 尊重网站的反爬虫机制
5. 仅用于合法目的

🔒 **安全注意**：
- 不要爬取需要登录的内容（除非获得授权）
- 不要绕过安全验证（如验证码）
- 不要对服务器造成过大压力
- 保护爬取到的数据，遵守隐私法规

---

**祝学习愉快！记住：做一个有礼貌、合规的爬虫开发者！** 🕷️
